<a href="https://colab.research.google.com/github/hby1117/class2025Spring/blob/main/08_AI_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio

In [ ]:
!pip install gradio
!pip install SpeechRecognition
!pip install gtts

In [ ]:
from gtts import gTTS
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
import speech_recognition as sr
import soundfile as sf

## Simple Calculator

In [ ]:
def add(first_num, second_num):
  result = first_num + second_num
  return f"Answer is: {result}"

interface = gr.Interface(fn=add, inputs = ["number", "number"], outputs="text")
interface.launch()

## Speech-to-Text (STT)

In [ ]:
def stt(audio_file):
    stt_model = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio_recorded = stt_model.record(source)
        text = stt_model.recognize_google(audio_recorded)
    return text

interface = gr.Interface(fn=stt, inputs="file", outputs="text")
interface.launch()

## Text-to-Speech (TTS)

In [ ]:
def tts(text):
  tts_model = gTTS(text)
  tts_model.save("tts_output.wav")
  return "tts_output.wav"

interface = gr.Interface(fn=tts, inputs="text", outputs="audio")
interface.launch()

## Voice Chatbot

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
chat_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

def stt(audio_file):
    stt_model = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio_recorded = stt_model.record(source)
        text = stt_model.recognize_google(audio_recorded)
    return text

def tts(text):
  tts_model = gTTS(text)
  tts_model.save("tts_output.wav")
  return "tts_output.wav"

# No chat history
def chatbot(text):
  input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")
  output_ids = chat_model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
  output = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
  return output

def voice_chatbot(audio):
  user_text = stt(audio)
  chatbot_text = chatbot(user_text)
  speech_file = tts(chatbot_text)
  return user_text, chatbot_text, speech_file

interface = gr.Interface(
    fn=voice_chatbot,
    inputs="file",
    outputs=[
        gr.Textbox(label="User Text"),
        gr.Textbox(label="Chatbot Response"),
        gr.Audio(label="Chatbot Audio")
    ]
)

interface.launch()